In [1]:
from newspaper import fulltext, Article
from datetime import date, timedelta
import requests
import pandas as pd
from dateutil import parser
import pandas as pd
import numpy as np
import time
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
start_date = date(2020,5,3)
end_date = date(2020,12,9)
dates = pd.date_range(start_date,end_date-timedelta(days=1),freq='d')

In [3]:
formatted_dates = []

In [4]:
for date in dates:
    date = date.strftime('%m-%d-%Y')
    formatted_dates.append(date)
    print(date)

05-03-2020
05-04-2020
05-05-2020
05-06-2020
05-07-2020
05-08-2020
05-09-2020
05-10-2020
05-11-2020
05-12-2020
05-13-2020
05-14-2020
05-15-2020
05-16-2020
05-17-2020
05-18-2020
05-19-2020
05-20-2020
05-21-2020
05-22-2020
05-23-2020
05-24-2020
05-25-2020
05-26-2020
05-27-2020
05-28-2020
05-29-2020
05-30-2020
05-31-2020
06-01-2020
06-02-2020
06-03-2020
06-04-2020
06-05-2020
06-06-2020
06-07-2020
06-08-2020
06-09-2020
06-10-2020
06-11-2020
06-12-2020
06-13-2020
06-14-2020
06-15-2020
06-16-2020
06-17-2020
06-18-2020
06-19-2020
06-20-2020
06-21-2020
06-22-2020
06-23-2020
06-24-2020
06-25-2020
06-26-2020
06-27-2020
06-28-2020
06-29-2020
06-30-2020
07-01-2020
07-02-2020
07-03-2020
07-04-2020
07-05-2020
07-06-2020
07-07-2020
07-08-2020
07-09-2020
07-10-2020
07-11-2020
07-12-2020
07-13-2020
07-14-2020
07-15-2020
07-16-2020
07-17-2020
07-18-2020
07-19-2020
07-20-2020
07-21-2020
07-22-2020
07-23-2020
07-24-2020
07-25-2020
07-26-2020
07-27-2020
07-28-2020
07-29-2020
07-30-2020
07-31-2020
08-01-2020

In [5]:
df = pd.DataFrame(columns= ['update', 'date', 'source', 'author', 'fulltext', 'summary'])

In [6]:
cleaned_links = []

In [7]:
def get_headlines(company, date_range):
    #global formatted_dates
    global cleaned_links
    global formatted_dates
    global df
    l = []
    ChromeOptions = webdriver.ChromeOptions()
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=ChromeOptions)
    
    for date in date_range:
        goog_news = "https://www.google.com"
        driver.get(goog_news)
        time.sleep(np.random.randint(1.45,2.35))
        search_bar = driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div[1]/div[1]/div/div[2]/input').send_keys(company)

        #press enter after typing in search term
        driver.find_element_by_css_selector('#tsf > div:nth-child(2) > div.A8SBwf > div.RNNXgb > div > div.a4bIc > input').send_keys(u'\ue007')

        #click on news
        time.sleep(np.random.randint(2,3))
        driver.find_element_by_link_text('News').click()

        #click on tools
        time.sleep(np.random.randint(1.02,2.35))
        driver.find_element_by_css_selector('#hdtb-tls').click()

        #click on recent
        time.sleep(np.random.randint(1.02,2.35))
        driver.find_element_by_css_selector('#hdtbMenus > div > span:nth-child(2) > g-popup > div.rIbAWc.hide-focus-ring > div > div').click()

        #click on custom range
        time.sleep(np.random.randint(1.02,2.35))
        driver.find_element_by_css_selector('#lb > div > g-menu > g-menu-item:nth-child(8) > div > div > span').click()

        #'from' input box
        time.sleep(np.random.randint(1.02,2.35))
        driver.find_element_by_css_selector('#OouJcb').send_keys(date)

        # 'to' input box
        time.sleep(np.random.randint(1.02,2.35))
        driver.find_element_by_css_selector('#rzG2be').send_keys(date)

        # 'go' button
        time.sleep(np.random.randint(1.02,2.35))
        driver.find_element_by_css_selector('#T3kYXe > g-button').click()


        while True:
            browser = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
            headers = {'User-Agent':browser,}
            time.sleep(np.random.randint(1.02,2.35))
            page = requests.get(str(driver.current_url))
            soup = BeautifulSoup(page.content, "lxml")
            links = soup.findAll("a")


            for link in  soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):
                l.append(re.split(":(?=http)",link["href"].replace("/url?q=",""))[0])
            try:
                elm = driver.find_element_by_css_selector('#pnnext > span.SJajHc.NVbCr')
                if 'inactive' in elm.get_attribute('class'):
                    break;
                elm.click()
                time.sleep(np.random.randint(1.02,2.35))
            except:
                break
        print('Scraped {} articles from {}'.format(company, date))
        #need to remove all chars after ampersand in the links
        #or else they break.
        print('cleaning links...')
        for link in l:
            res = re.sub(r'&.*',"",link)
            cleaned_links.append(res)
        #adding the article info to the dataframe.
        print('compiling dataframe...')
        for link in cleaned_links:
            try:
                article= Article(link)
                article.download()
                article.parse()
                text = article.text
                published = article.publish_date
                auth = article.authors
                source = link
                summ = article.summary
                titl = article.title
                df = df.append({'update':published, 'date':date, 'source':link, 'author':auth, 'fulltext':text, 'summary':summ, 'title':titl}, ignore_index=True)
            except:
                continue
        df.to_csv('output.csv')
        cleaned_links = []
        l = []
        print("cleaned list: {}".format(len(cleaned_links)))
        print("length l: {}".format(len(l)))
        print(df.shape)   
    driver.close()
    return 'Done scraping, {} links.'.format(len(l))
        

In [ ]:
get_headlines('Apple', formatted_dates)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/brendanferris/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


 
Scraped Apple articles from 05-03-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(20, 7)
Scraped Apple articles from 05-04-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(80, 7)
Scraped Apple articles from 05-05-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(140, 7)
Scraped Apple articles from 05-06-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(218, 7)
Scraped Apple articles from 05-07-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(296, 7)
Scraped Apple articles from 05-08-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(336, 7)
Scraped Apple articles from 05-09-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(376, 7)
Scraped Apple articles from 05-10-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(396, 7)
Scraped Apple articles from 05-11-2020
cleaning links...
compili

cleaned list: 0
length l: 0
(3322, 7)
Scraped Apple articles from 07-12-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(3342, 7)
Scraped Apple articles from 07-13-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(3382, 7)
Scraped Apple articles from 07-14-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(3442, 7)
Scraped Apple articles from 07-15-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(3482, 7)
Scraped Apple articles from 07-16-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(3522, 7)
Scraped Apple articles from 07-17-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(3562, 7)
Scraped Apple articles from 07-18-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(3582, 7)
Scraped Apple articles from 07-19-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(3602, 7)
Scraped Apple arti

cleaned list: 0
length l: 0
(6138, 7)
Scraped Apple articles from 09-20-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(6158, 7)
Scraped Apple articles from 09-21-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(6198, 7)
Scraped Apple articles from 09-22-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(6238, 7)
Scraped Apple articles from 09-23-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(6278, 7)
Scraped Apple articles from 09-24-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(6318, 7)
Scraped Apple articles from 09-25-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(6358, 7)
Scraped Apple articles from 09-26-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(6378, 7)
Scraped Apple articles from 09-27-2020
cleaning links...
compiling dataframe...
cleaned list: 0
length l: 0
(6398, 7)
Scraped Apple arti

In [ ]:
df

# Getting Newspaper Articles

In [ ]:
df = pd.DataFrame(columns= ['update', 'date', 'source', 'author', 'fulltext'])

In [ ]:
fulltxt = []

In [ ]:
for link in cleaned_links:
    try:
        article= Article(link)
        article.download()
        article.parse()
        text = article.text
        published = article.publish_date
        auth = article.authors
        source = link
        df = df.append({'update':published, 'source':link, 'author':auth, 'fulltext':text}, ignore_index=True)
    except:
        continue

In [ ]:
df

In [ ]:
df.iloc[2]['source']

In [ ]:
article = Article(l[84], language='en')

In [ ]:
article.download()

In [ ]:
cleaned_links = []
for link in l:
    res = re.sub(r'&.*',"",link)
    cleaned_links.append(res)
    

In [ ]:
article = Article(cleaned_links[16], language='en')

In [ ]:
article.download()
article.parse()